### 데이터불러오기

In [6]:
#데이터 불러오기
import pandas as pd
df1 = pd.read_csv("train.csv", engine = 'python')
df2 = pd.read_csv("test.csv", engine = 'python')
submit = pd.read_csv("submit.csv",engine = 'python')

In [7]:
#train data
X = df1.drop(['label'],axis=1)
y = df1['label']

X_test = df2
X_test.head()

,N,P,K,temperature,humidity,ph,rainfall
0,101,17,47,29.494014,94.729813,6.185053,26.308209
1,98,8,51,26.179346,86.522581,6.259336,49.430510
2,59,62,49,43.360515,93.351916,6.941497,114.778071
3,44,60,55,34.280461,90.555616,6.825371,98.540477
4,30,137,200,22.914300,90.704756,5.603413,118.604465


### 입력데이터 실수화

In [8]:
#데이터셋 카테고리의 실수화
from sklearn.preprocessing import LabelEncoder
import numpy as np
classle = LabelEncoder()
y = classle.fit_transform(df1['label'].values) #문자열을 categorial 값으로 전환
print('species label:',np.unique(y)); #unique:중복 제외

species label: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


### 데이터분할

In [9]:
#데이터분할
#validation 생성
#from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.3, random_state = 1, stratify = y)

X_train = X
y_train = y

##### 표준화

In [10]:
#입력데이터 표준화
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
#X_val_std = sc.transform(X_val)
X_test_std = sc.transform(X_test)

### 모델추정 및 사례중심학습

#### 표준화x

In [29]:
#모델추정 및 사례중심학습
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7, p = 1)
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=1,
                     weights='uniform')

In [30]:
y_train_pred = knn.predict(X_train)
#y_val_pred = knn.predict(X_val)
y_test_pred = knn.predict(X_test)

print('MisClassified training samples: %d'%(y_train!=y_train_pred).sum())
#print('MisClassified validation samples: %d'%(y_val!= y_val_pred).sum())

MisClassified training samples: 16


In [18]:
#결과분석
#from sklearn.metrics import accuracy_score
#print(accuracy_score(y_val,y_val_pred))

In [17]:
#from sklearn.metrics import confusion_matrix #오분류표 작성을 위한 모듈 import
#conf=confusion_matrix(y_true=y_val,y_pred=y_val_pred)
#print(conf)

#### 표준화

In [36]:
knn.fit(X_train_std,y_train)
y_train_std_pred = knn.predict(X_train_std)
y_test_std_pred = knn.predict(X_test_std)
#y_val_std_pred = knn.predict(X_val_std)

print('MisClassified training samples: %d'%(y_train!=y_train_std_pred).sum())
#print('MisClassified validation samples: %d'%(y_val!= y_val_pred).sum())

MisClassified training samples: 21


In [37]:
#결과분석
#from sklearn.metrics import accuracy_score
#print(accuracy_score(y_val,y_val_std_pred))

In [38]:
#from sklearn.metrics import confusion_matrix #오분류표 작성을 위한 모듈 import
#conf=confusion_matrix(y_true=y_val,y_pred=y_val_std_pred)
#print(conf)

### 마할노비스거리

In [157]:
knn_mahal=KNeighborsClassifier(metric='mahalanobis', metric_params={'V':np.cov(X_train, rowvar=False)})
knn_mahal.fit(X_train, y_train)

y_train_mahal_pred=knn_mahal.predict(X_train)
y_val_mahal_pred=knn_mahal.predict(X_val)

print(accuracy_score(y_train, y_train_mahal_pred))
print(accuracy_score(y_val, y_val_mahal_pred))

0.9835497835497835
0.9616161616161616


In [158]:
#표준화
knn_mahal.fit(X_train_std, y_train)
y_train_mahal_pred=knn_mahal.predict(X_train_std)
y_val_mahal_pred=knn_mahal.predict(X_val_std)

print(accuracy_score(y_train, y_train_mahal_pred))
print(accuracy_score(y_val, y_val_mahal_pred))

0.7489177489177489
0.5616161616161616


### submit 제출

In [33]:
y_test_pred = classle.inverse_transform(y_test_pred) #원래의 문자열로 전환
print('species:',np.unique(y_test_pred))

species: ['apple' 'banana' 'blackgram' 'chickpea' 'coconut' 'coffee' 'cotton'
 'grapes' 'jute' 'kidneybeans' 'lentil' 'maize' 'mango' 'mothbeans'
 'mungbean' 'muskmelon' 'orange' 'papaya' 'pigeonpeas' 'pomegranate'
 'rice' 'watermelon']


In [34]:
submit['label']=y_test_pred
submit

,id,label
0,0,muskmelon
1,1,watermelon
2,2,papaya
3,3,papaya
4,4,apple
...,...,...
545,545,coffee
546,546,papaya
547,547,chickpea
548,548,coffee


In [35]:
import pandas as pd
submit.to_csv('submit.csv', index=False, encoding='cp949')